# Cadastral data mining

- The first step is reading relevant data to be processed. We store all .CAT files in the same directory

In [91]:
#Leemos el Notebook de Utils
#We read the Utils Notebook
%run Utils.ipynb
plt.style.use('fivethirtyeight')

In [92]:
path = "C:\\Users\\manub\\DATA\\00-EXTERNO\\MANU\\CARPIO\\"

In [93]:
cat_file = path+"28_900_U_2021-01-22.CAT"
parcels_file = path+"28_900_UH_2021-01-23_SHF\\PARCELA\\PARCELA.SHP"
#access_file = path+"MADRID\\28_900_UA_2020-07-31_SHF\\ELEMTEX\\ELEMTEXII.SHP"
#network_file = path + "MADRID\\RT_MADRID_INTEREST.shp"

# Work on cadastral .cat
It is a structured text file which description can be obtained [here](http://www.catastro.minhap.es/documentos/formatos_intercambio/catastro_fin_cat_2006.pdf)

In [94]:
df11 = prepare_df(leecat11(cat_file))
df13 = prepare_df(leecat13(cat_file))
df14 = prepare_df(leecat14(cat_file))
df15 = prepare_df(leecat15(cat_file))

Leyendo CAT C:\Users\manub\DATA\00-EXTERNO\MANU\CARPIO\28_900_U_2021-01-22.CAT: Registro 11
Hecho
Leyendo CAT C:\Users\manub\DATA\00-EXTERNO\MANU\CARPIO\28_900_U_2021-01-22.CAT: Registro 13
Hecho
Leyendo CAT C:\Users\manub\DATA\00-EXTERNO\MANU\CARPIO\28_900_U_2021-01-22.CAT: Registro 14
Hecho
Leyendo CAT C:\Users\manub\DATA\00-EXTERNO\MANU\CARPIO\28_900_U_2021-01-22.CAT: Registro 15
Hecho


In [95]:
cityname = df11['NOMMUN'].iloc[0]
print("Reconocido municipio: "+cityname)

Reconocido municipio: MADRID


In [96]:
df11 = df11[['PCAT','CODPROV','CODINE','NOMPROV','NOMMUN','PCODVIA','PTIPVIA','PNOMVIA','PPNP','PPLE','SUPP','SUPCT','SUPCSR','SUPCBR','SUPCUB','COORX','COORY','DEN_BICE']]
df13 = df13[['PCAT','COD_UC','ANOCONS','ANOEXAC']]
df14 = df14[['PCAT','NCBI_PC','COD_UC','CDD','TIPREF','ANOREF','ANOEFE','SUPT_CAT']]
#Drop common spaces
df14 = df14.loc[df14['NCBI_PC'] != ""]
df15 = df15[['PCAT','NCBI_PC','CODVIA','TIPVIA','NOMVIA','PNP','PLE','NOBI','ANOANT','CDD_BI','SUPCON']]

In [97]:
#Merge all four register tables
df = merge_and_drop(df15,df14,'right',['PCAT','NCBI_PC'])
df = merge_and_drop(df13,df,'right',['PCAT','COD_UC'])
df = merge_and_drop(df11,df,'right',['PCAT'])
df['PPNP'] = df['PPNP'].fillna(0).astype('int64').astype('str').str.replace("0", "")
df['PNP'] = df['PNP'].fillna(0).astype('int64').astype('str').str.replace("0", "")

In [98]:
len(df.PCAT.unique())

122484

In [99]:
df_bi = df.drop_duplicates(subset = ['PCAT','NCBI_PC'],keep='first')

In [100]:
sums = pd.pivot_table(df_bi, values='SUPCON', index=['PCAT'],columns=['CDD_BI'], aggfunc=np.sum).fillna(0)

In [101]:
counts = pd.pivot_table(df_bi, values='SUPCON', index=['PCAT'],columns=['CDD_BI'], aggfunc=lambda x: len(x.unique())).fillna(0)

In [102]:
sums.rename(columns = {c:"S"+c for c in sums.columns},inplace=True)

In [103]:
counts.rename(columns = {c:"N"+c for c in counts.columns},inplace=True)

In [104]:
parcels = gpd.read_file(parcels_file)
parcels = parcels.dissolve(by = 'REFCAT')
parcels = parcels.loc[~parcels['geometry'].isna()]

In [105]:
parcels_loaded = pd.merge(parcels,counts,left_index=True,right_index=True,how='left')
parcels_loaded = pd.merge(parcels,sums,left_index=True,right_index=True,how='left')

In [106]:
for p in [c for c in parcels_loaded.columns if len(c)==2]:
    parcels_loaded[p].fillna(0,inplace=True)

In [107]:
len(parcels_loaded)

159141

In [108]:
parcels_loaded.to_file(path+cityname+"_loaded.shp",driver='ESRI Shapefile')

# Get Land Use Reference points ready
Basically, I use cadastral addresses points described [here](http://www.catastro.minhap.es/ayuda/manual_descriptivo_shapefile.pdf), assign the nearest parcel id to each one,and then try to match them to my cadastral records. This point is the closest I can get to the actual entrance of each land use activity, I call it ***'refpoint'***

In [ ]:
addresses = gpd.read_file(access_file)
#addresses = addresses.loc[addresses['INTEREST']==1]
addresses = addresses.loc[~addresses['geometry'].isna()]
parcels = gpd.read_file(parcels_file)
#parcels = parcels.loc[parcels['INTEREST']==1]
parcels = parcels.loc[~parcels['geometry'].isna()]

In [ ]:
#Get the address points
addresses = addresses.loc[addresses['TTGGSS'] == '189401']
addresses['geometry'] = addresses['geometry'].centroid

In [ ]:
addresses.head(2)

In [ ]:
parcels.head(2)

In [ ]:
fig, ax = plt.subplots(figsize =(10,10))
ax.set_facecolor('black')
parcels.plot(ax=ax)
addresses.plot(ax=ax,
                    color='lightgray',
                    markersize = 2)
plt.grid(True,color='w', linestyle='-', linewidth=0.3)
plt.show();
plt.clf()

In [ ]:
%run Utils.ipynb

#Find Nearest Parcels
addresses = find_nearest(addresses,parcels,'REFCAT')[['ROTULO','REFCAT','geometry']]
display(addresses.head())

In [ ]:
#Three alternatives (Proper Address, Parcel Address, Parcel Centroid)
addresses['gaddress'] = addresses['REFCAT']+addresses['ROTULO']
addresses = addresses[['REFCAT','ROTULO','gaddress','geometry']]
df['address'] = df['PCAT']+df['PNP']+df['PLE']
df['paddress'] = df['PCAT']+df['PPNP']+df['PPLE']
dict_address_points = pd.Series(addresses.geometry.values,index=addresses.gaddress).to_dict()
df['refpoint'] = df['address'].map(dict_address_points)
dfa = df[df['refpoint'].notnull()]
dfb = df[df['refpoint'].isnull()]
dfb['refpoint'] = dfb['paddress'].map(dict_address_points)
dfc = dfb[dfb['refpoint'].isnull()]             
dfb = dfb[dfb['refpoint'].notnull()]
dfc['refpoint'] = dfc.apply(lambda row: Point(row.COORX, row.COORY), axis=1) 
reference_points = pd.concat([dfa,dfb,dfc])
reference_points['geometry'] = reference_points['refpoint'] 
display(reference_points.head())

# Process network data and find possible network reference points for land use

Now I need another point where my addresses (entrances) snap to the network, in order to create a significant point of activity along my street network edges. I create a set of possible measure points and snap my addresses to it

In [ ]:
network = gpd.read_file(network_file)
#network = network.loc[network['INTEREST2']==1]

In [ ]:
#Create measure points by splitting original network into 50 m chunks and retrieving all the resulting line intersections
spl = lines_and_interpolated_vertices(network,100,['id_vial','id_tramo','tipo_vial','clase','firme','tipovehic'])
split_interpolated_network = spl[0]
modified_network = spl[1]

In [ ]:
measure_points = get_lines_endpoints(split_interpolated_network)
measure_points = gpd.GeoDataFrame(measure_points,columns=['geometry'])
measure_points['netpoint'] = measure_points.index
display(measure_points.head())

In [ ]:
print(len(reference_points))

In [ ]:
#Find Nearest Parcels
final_registers = find_nearest(reference_points,measure_points,'netpoint')
final_registers = pd.merge(final_registers,measure_points,how='left',on='netpoint')
final_registers['netpointg'] = final_registers['geometry_y']
final_registers.drop(columns=['geometry_x','geometry_y'],inplace=True)

# Prepare final network geometry

Now I want to create edges geometries that are aware of all intersections in my original data, as well as the identified network reference points, so I optimize the number of edges used

First, I will grab all the intersections of the original data

In [ ]:
network['geometry'] = network['geometry'].astype('str')
original_endpoints = [p.wkt for p in get_lines_endpoints(network)]
points_with_activity =  [p.wkt for p in list(final_registers.netpointg.unique())]
final_network_points = list(set(original_endpoints+points_with_activity))

Then, I will create a network that is split on the new interpolated points

In [ ]:
final_split_network = split_lines_at_points(modified_network,final_network_points,['id_vial','id_tramo','tipo_vial','clase','firme','tipovehic'],'id_tramo')

In [ ]:
final_split_network = calculate_speed_by_slope(final_split_network,'length','slope')

Finally, I need to referr my cadastral registers to the final network points, as the set has changed significantly. I will recreate the index column and relate my reference points to these again

In [ ]:
final_network_points = pd.DataFrame(final_network_points)
final_network_points['netpoint'] = final_network_points.index

In [ ]:
final_registers.drop(columns=['netpoint','netpointg'],inplace=True)

In [ ]:
display(final_registers.head())

In [ ]:
final_registers['geometry'] = final_registers['refpoint']
final_network_points.rename(columns = {0:'geometry'},inplace = True)

In [ ]:
final_network_points['geometry'] = final_network_points.apply(lambda row: Point(loads(row['geometry'])),axis=1)

In [ ]:
final_network_points.to_csv(path+"\\MADRID\\final_network_points.csv")

In [ ]:
final_network_points = gpd.GeoDataFrame(final_network_points,geometry ='geometry')

In [ ]:
final_registers = find_nearest(final_registers,final_network_points,'netpoint')

In [ ]:
final_registers.rename(columns = {'geometry':'netpointg'},inplace=True) 

In [ ]:
final_registers.to_csv(path+"\\MADRID\\CAT_referenciado.csv")
final_network_points.to_csv(path+"\\MADRID\\RED_nodos.csv")

In [ ]:
measure_points.to_csv(path+"\\MADRID\\measure_points.csv")

In [ ]:
final_split_network['geometry'] = final_split_network['geometry'].apply(loads)

final_split_network = gpd.GeoDataFrame(final_split_network,geometry='geometry')
final_split_network.to_file(path+"\\MADRID\\edges_loaded.shp",driver = 'Shapefile')

In [ ]:
final_split_network.to_csv(path+"\\MADRID\\edges_loaded.csv")